In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import datetime
from pprint import pprint as pp
import sys
import pickle
import glob
import os

sys.path.insert(0, '/home/molly/Desktop/DeepTCGA/')
import load_data

/home/molly/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def fc_layer(A_prev, size_in, size_out, name="fully-connected"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], mean=0, stddev=0.1))
        b = tf.Variable(tf.constant(0.1, shape=[size_out]))
        act = tf.matmul(A_prev, w) + b
        return act, w, b

    
def build_model(x, N_IN, N_HIDDEN):
    parameters = {}
    z1, w1, b1 = fc_layer(x, N_IN, N_HIDDEN, name="fc1")
    parameters.update({"z1":z1, "w1": w1, "b1": b1})
    hidden = tf.nn.leaky_relu(z1, name="hidden")
    x_recon, w2, b2 = fc_layer(hidden, N_HIDDEN, N_IN, name="fc2")
    parameters.update({"w2": w2, "b2": b2})
    return x_recon, parameters


def back_prop(x, x_recon, learning_rate):
    loss = tf.reduce_mean(tf.square(x_recon - x))
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return train_step


def feed_forward(x, parameters):
    w1, b1 = parameters["w1"], parameters["b1"]
    hidden = tf.nn.leaky_relu(tf.matmul(x, w1) + b1)
    w2, b2 = parameters["w2"], parameters["b2"]
    x_recon = tf.matmul(hidden, w2) + b2
    return x_recon, hidden


def mse(x, x_recon, name=""):
    mse = tf.reduce_mean(tf.square(x_recon-x))
    return mse


def create_folder(result_folder):
    assert(result_folder!="")
    path = "../results/AE/{0}/".format(result_folder)
    if len(glob.glob(path)) == 0:
        os.mkdir(path)
    return path

In [3]:
def train_model(data, n_hidden, batch_size=128, 
                num_epoch=1000, learning_rate=1e-3, result_folder=""):
    tf.reset_default_graph()
    folder = create_folder(result_folder)
    N_IN = data.train.num_features
    N_OUT = data.train.num_features
    N_HIDDEN = n_hidden
    
    # train step
    (train_batch, train_iter, val_all, val_iter, 
        train_all, train_iter_all) = data.prep_batch(batch_size=batch_size, 
                                                     count_by="epoch")
    x = train_batch["X"]
    x_recon, parameters = build_model(x, N_IN, N_HIDDEN)
    train_step = back_prop(x, x_recon, learning_rate)
    
    # mse
    x_train, x_val = train_all["X"], val_all["X"]
    x_train_recon, train_latent = feed_forward(x_train, parameters)
    x_val_recon, val_latent = feed_forward(x_val, parameters)
    train_mse = mse(x_train, x_train_recon, name="train")
    val_mse = mse(x_val, x_val_recon, name="valiation")
    
    # run
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    train_errors = []
    val_errors = []
    try:
        for epoch in range(num_epoch):
            sess.run([train_iter.initializer])
            t0 = time.time()
            try:
                while True:
                    sess.run(train_step)
            except tf.errors.OutOfRangeError:
                sess.run([train_iter_all.initializer, val_iter.initializer])
                [train_error, val_error] = sess.run([train_mse, val_mse])
                train_errors.append(train_error)
                val_errors.append(val_error)
                if epoch % 10 == 0:
                    print("epoch", epoch)
                    print("training mse:", train_error, "validation mse", val_error)
                    print("epoch time:", time.time()-t0)
    finally:
        train_latent = sess.run(train_latent)
        val_latent = sess.run(val_latent)
        parameter_trained = sess.run(parameters)
        parameter_trained["total_epoch"] = epoch
        np.save("{0}/train_latent.npy".format(folder), train_latent)
        np.save("{0}/val_latent.npy".format(folder), val_latent)
        with open("{0}/parameters.pkl".format(folder), "wb") as f:
            pickle.dump(parameter_trained, f)
        sess.close()

In [4]:
tcga = load_data.read_data_sets("../data/mRNA_lognorm_StandardScaled.hdf")

In [ ]:
for node in [8000, 4000, 2000, 1000, 500, 250, 100, 50, 20, 10, 2]:
    train_model(tcga, node, num_epoch=2000, result_folder=str(node))

epoch 0
training mse: 513.3475 validation mse 619.1425
epoch time: 7.027270317077637
epoch 10
training mse: 88.10996 validation mse 224.1966
epoch time: 5.287370920181274
epoch 20
training mse: 43.066147 validation mse 176.27597
epoch time: 5.3054211139678955
epoch 30
training mse: 40.137268 validation mse 170.9424
epoch time: 5.288269996643066
epoch 40
training mse: 34.099155 validation mse 163.63919
epoch time: 5.301032781600952
epoch 50
training mse: 21.52984 validation mse 146.24998
epoch time: 5.261935234069824
epoch 60
training mse: 18.929178 validation mse 144.52551
epoch time: 5.287148714065552
epoch 70
training mse: 17.108112 validation mse 138.10272
epoch time: 5.293067216873169
epoch 80
training mse: 11.502903 validation mse 126.5247
epoch time: 5.3004841804504395
epoch 90
training mse: 7.6458073 validation mse 114.50571
epoch time: 5.287476301193237
epoch 100
training mse: 6.513771 validation mse 108.278435
epoch time: 5.311220169067383
epoch 110
training mse: 4.6273785 val